In [ ]:
import pandas as pd
import numpy as np
import jieba
import os
import jieba.posseg as pseg
if not os.path.exists('data/dict.big.txt'):
    os.system("wget %s -O %s" % (
        'https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big',
        'data/dict.big.txt'))
jieba.set_dictionary('data/dict.big.txt')

In [ ]:
# Preprocessing (seg)
# Comebine test data & review 
test_data = pd.read_csv('data/test.csv')
review = {}
with open('data/test_review.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    for i in range(0, len(lines), 2):
        review[int(lines[i])] = ' '.join([tok for tok in jieba.cut(lines[i+1])])

test_data['Review'] = [review[review_id] for review_id in test_data.Review_id]
test_data.to_csv('data/test_seg.csv', index=None)

# Seg polarity review
review, polarity = [], []
with open('data/polarity_review.txt') as f:
    for line in [line.strip() for line in f.readlines()]:
        l = line.split('\t')
        polarity.append(int(l[0]))
        review.append(' '.join([tok for tok in jieba.cut(l[1])]))
polarity_review_seg = pd.DataFrame({'Polarity': polarity, 'Review': review})
polarity_review_seg.to_csv('data/polarity_review_seg.csv', index=None)

# Seg aspect review
aspect_review = []
with open('data/aspect_review.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    for i in range(0, len(lines), 4):
        aspect_review.append({
                'Id': int(lines[i]),
                'Review': ' '.join([tok for tok in jieba.cut(lines[i+1])]),
                'Pos': lines[i+2],
                'Neg': lines[i+3]
            })
aspect_review = pd.DataFrame(aspect_review)
aspect_review.to_csv('data/aspect_review_seg.csv', index=None)

In [ ]:
# Pos & seg
def posseg(s):
    s = s.replace(' ', '')
    tokens = pseg.cut(s)
    return '\n'.join(['%s %s' % (token.word, token.flag) for token in tokens])

test_data.Review = test_data.Review.apply(posseg)
test_data.to_csv('data/test_posseg.csv', index=None)

polarity_review_seg.Review = polarity_review_seg.Review.apply(posseg)
polarity_review_seg.to_csv('data/polarity_review_posseg.csv', index=None)

aspect_review.Review = aspect_review.Review.apply(posseg)
aspect_review.to_csv('data/aspect_review_posseg.csv', index=None)

In [ ]:
aspect_review = pd.read_csv('data/aspect_review_seg.csv')
polarity_review = pd.read_csv('data/polarity_review_seg.csv')
test_review = pd.read_csv('data/test_seg.csv')
test_review = test_review.groupby('Review_id').first()
reviews = aspect_review['Review'].append(polarity_review['Review']).append(test_review['Review'])
print('#reviews:', len(reviews))

if os.path.exists('data/external_corpus.txt'):
    print('using extrenal corpus, it may take pretty loooong')
    with open('data/external_corpus.txt', "r") as inputs:
        with open('data/extrenal_corpus.txt', 'w') as cutted:
            for line in inputs.readlines():
                cutted.write(jieba.cut(line))
else:
    print('external corpus dosen\'t exitst (put it as data/external_corpus.txt plz QAQ)')
    

with open("data/word2vec_corpus.tmp", "w") as f:
    if os.path.exists('data/external_corpus_seg.txt'):
        with open("data/external_corpus_seg.txt") as ext:
            f.write(ext.read())
        f.write(("\n".join(reviews)+"\n")*5)
    else:
        f.write(("\n".join(reviews)))

In [ ]:
if os.path.exists('word2vec/word2vec'):
    printf('running word2vec ...')
    os.system('time word2vec/word2vec -train data/word2vec_corpus.tmp -output vectors.bin -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 20 -binary 1 -iter 15')
else:
    printf('word2vec doesn\'t exitst (put it as word2vec/word2vec plz QAQ)')